In [1]:
import numpy as np 
import pandas as pd

In [2]:
BASE_PATH = r"D:\DATA_ANALYST\FULL_STACK_FROJECT\Stock Market Prediction\stock_market_unclean_dataset"
df = pd.read_csv(BASE_PATH + r"\trading_data\volumes.csv")


In [3]:
df.head()

,date,company,volume
0,2015-01-01,HDFC,1719741
1,2015-01-02,HDFC,1905072
2,2015-01-05,HDFC,2395398
3,2015-01-06,HDFC,1583246
4,2015-01-07,HDFC,2395040


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144600 entries, 0 to 144599
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   date     144600 non-null  object
 1   company  144600 non-null  object
 2   volume   144600 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 3.3+ MB


In [5]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df.dropna(subset=['date'], inplace=True)
df.sort_values(by=['company', 'date'], inplace=True)

In [6]:
df['company'] = df['company'].str.strip().str.upper()

In [7]:
df = df[df['volume'] >= 0]

In [8]:
df.drop_duplicates(subset=['date', 'company'], inplace=True)

In [9]:
df = df[df['volume'] < df['volume'].quantile(0.999)]

In [10]:
df.reset_index(drop=True, inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144455 entries, 0 to 144454
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   date     144455 non-null  datetime64[ns]
 1   company  144455 non-null  object        
 2   volume   144455 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 3.3+ MB


In [12]:
df.head()

,date,company,volume
0,2015-01-01,ADANIPOWER,2147267
1,2015-01-02,ADANIPOWER,1814073
2,2015-01-05,ADANIPOWER,2045727
3,2015-01-06,ADANIPOWER,2375882
4,2015-01-07,ADANIPOWER,2056699


In [13]:
df['avg_volume_7d'] = (df.groupby('company')['volume'].rolling(7).mean().reset_index(level=0, drop=True))

In [14]:
df['volume_spike'] = df['volume'] > df['avg_volume_7d']

In [15]:
df.dropna(subset=['avg_volume_7d'], inplace=True)

In [16]:
df.head()

,date,company,volume,avg_volume_7d,volume_spike
6,2015-01-09,ADANIPOWER,1432284,2.028793e+06,False
7,2015-01-12,ADANIPOWER,1631393,1.955096e+06,False
8,2015-01-13,ADANIPOWER,1738258,1.944266e+06,False
9,2015-01-14,ADANIPOWER,2054027,1.945451e+06,True
10,2015-01-15,ADANIPOWER,1446502,1.812683e+06,False


In [17]:
df['avg_volume_7d_million'] = df['avg_volume_7d'] / 1_000_000

In [18]:
df['avg_volume_7d_million'] = df['avg_volume_7d_million'].round(2)

In [19]:
df.head()

,date,company,volume,avg_volume_7d,volume_spike,avg_volume_7d_million
6,2015-01-09,ADANIPOWER,1432284,2.028793e+06,False,2.03
7,2015-01-12,ADANIPOWER,1631393,1.955096e+06,False,1.96
8,2015-01-13,ADANIPOWER,1738258,1.944266e+06,False,1.94
9,2015-01-14,ADANIPOWER,2054027,1.945451e+06,True,1.95
10,2015-01-15,ADANIPOWER,1446502,1.812683e+06,False,1.81


In [20]:
df = df.rename(columns={
    "date": "trade_date"
})
df = df[["company",
         "trade_date",
         "volume",
         "avg_volume_7d",
         "avg_volume_7d_million",
         "volume_spike"]]


In [21]:
df.to_csv("volumes.csv", index=False)